<a href="https://colab.research.google.com/github/kabulezhoushen/blender-mcp/blob/main/%E6%B3%A2%E5%8A%A8%E7%8E%87%E8%82%A1%E6%B1%A0%E6%9E%84%E5%BB%BA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#资讯科技业，金融业， 医疗保健业，非必需性消费，工业（备选）
#医疗40-60 资讯科技业60-80 金融业50-70 非必需性消费50-70 工业（备选）30-40
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 读取数据
hsi_data = pd.read_csv('hsi all shares data 20250613.csv', encoding='gbk')
stock_codes_df = pd.read_csv('stock_codes_full (1).csv')
print(hsi_data)
print(stock_codes_df)

# 获取医疗保健业股票代码
code_list_yiliao = stock_codes_df["医疗保健业"].dropna().tolist()
print(code_list_yiliao)

# 获取过去30天的成交量数据
data = yf.download(code_list_yiliao, period='30d', progress=False)
volume = data['Volume']
volume_mean = volume.mean()
print(volume_mean)

# 筛选出成交量排名前70%的股票代码
volume_mean_sorted = volume_mean.sort_values(ascending=False)
top_70_percent_count = int(len(volume_mean_sorted) * 0.7)
top_70_percent_codes = volume_mean_sorted.head(top_70_percent_count).index.tolist()
print(f"成交量排名前70%的股票代码: {top_70_percent_codes}")

# 在hsi all shares data 20250613.csv中获取这些股票的估值数据
filtered_data = hsi_data[hsi_data['WindCodes'].isin(top_70_percent_codes)].copy()

# 过滤掉极端高/低PE（pe_ttm）的股票
filtered_data = filtered_data[(filtered_data['PE_TTM'] > 0) & (filtered_data['PE_TTM'] < 100)]

# 剔除市值过小的股票（如总市值>50亿港币，自由流通市值>10亿港币）
filtered_data = filtered_data[(filtered_data['MKT_CAP'] > 5000000000) & (filtered_data['MKT_FREESHARES'] > 1000000000)]

# 计算营业收入和净利润的同比增长率
filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'] / filtered_data['WGSD_SALES'].shift(1) - 1
filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'] / filtered_data['WGSD_NET_INC'].shift(1) - 1

# 筛选增长为正或高于行业/市场中位数的股票
filtered_data = filtered_data[(filtered_data['SALES_GROWTH'] > 0) | (filtered_data['SALES_GROWTH'] > filtered_data['SALES_GROWTH'].median())]
filtered_data = filtered_data[(filtered_data['NET_PROFIT_GROWTH'] > 0) | (filtered_data['NET_PROFIT_GROWTH'] > filtered_data['NET_PROFIT_GROWTH'].median())]

# 筛选营业利润为正的公司，剔除亏损企业
filtered_data = filtered_data[filtered_data['WGSD_EBIT_OPER'] > 0]

# 计算每只股票的综合估值得分
filtered_data['score'] = (filtered_data['PE_TTM'] + filtered_data['MKT_CAP'] + filtered_data['MKT_FREESHARES'] + filtered_data['WGSD_SALES']) / 4

# 排序股票的综合估值得分
sorted_data = filtered_data.sort_values(by='score', ascending=False)

# 计算前15%的股票数量
top_15_percent_count = int(len(sorted_data) * 0.15)

# 获取综合估值得分排名前15%的股票代码
top_15_percent_codes = sorted_data.head(top_15_percent_count)['WindCodes'].tolist()

# 打印结果
print(f"综合估值得分排名前15%的股票共有 {len(top_15_percent_codes)} 只:")
print(top_15_percent_codes)

# 将结果存储在名为 top_15_value_codes 的新变量中
top_15_value_codes = top_15_percent_codes

# 可选：保存结果到csv
sorted_data.head(top_15_percent_count).to_csv('top_15_value_codes.csv', index=False, encoding='utf-8-sig')

      WindCodes     PE_TTM       MKT_CAP  MKT_FREESHARES    WGSD_SALES  \
0       0001.HK  10.545622  1.802036e+11    1.253513e+11  2.813510e+11   
1       0002.HK  14.222312  1.669984e+11    5.102448e+10  9.096400e+10   
2       0003.HK  22.052723  1.259541e+11    7.365045e+10  5.547280e+10   
3       0004.HK -21.801683  7.028863e+10    2.213811e+10  1.211500e+10   
4       0005.HK  10.606696  1.622476e+12    1.372409e+12  6.419200e+10   
...         ...        ...           ...             ...           ...   
2631    9996.HK -17.472189  4.274954e+09    1.810134e+09  6.154830e+08   
2632    9997.HK  15.256829  9.579392e+09    2.796225e+09  1.008610e+09   
2633    9998.HK   7.931432  1.520000e+08    3.800000e+07  1.359790e+08   
2634    9999.HK  18.824600  6.602027e+11    3.668847e+11  1.052952e+11   
2635  数据来源：Wind        NaN           NaN             NaN           NaN   

      WGSD_EBIT_OPER  WGSD_NET_INC  
0       2.477800e+10  1.708800e+10  
1       1.490300e+10  1.174200e+10  


/tmp/ipython-input-13-2591903107.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(code_list_yiliao, period='30d', progress=False)


Ticker
0013.HK    1.004241e+07
0239.HK    4.560000e+04
0241.HK    9.747565e+07
0286.HK    0.000000e+00
0314.HK    1.760656e+07
               ...     
9969.HK    1.897505e+07
9989.HK    3.601650e+06
9995.HK    1.110533e+07
9996.HK    1.973786e+06
9997.HK    4.975181e+06
Length: 253, dtype: float64
成交量排名前70%的股票代码: ['1093.HK', '0460.HK', '1177.HK', '2186.HK', '2228.HK', '0241.HK', '1530.HK', '1951.HK', '0512.HK', '2269.HK', '0876.HK', '1548.HK', '1302.HK', '1801.HK', '0690.HK', '0570.HK', '6955.HK', '0853.HK', '2096.HK', '0867.HK', '2157.HK', '9969.HK', '2005.HK', '0314.HK', '2616.HK', '3681.HK', '2453.HK', '9926.HK', '2367.HK', '9688.HK', '3692.HK', '1066.HK', '2158.HK', '3933.HK', '2172.HK', '2393.HK', '9885.HK', '9995.HK', '1558.HK', '6618.HK', '0928.HK', '2142.HK', '3320.HK', '0013.HK', '2252.HK', '1833.HK', '0775.HK', '1477.HK', '3759.HK', '6127.HK', '2607.HK', '1877.HK', '2196.HK', '1228.HK', '2522.HK', '1167.HK', '1276.HK', '1349.HK', '0719.HK', '6086.HK', '1099.HK', '6160.HK', '2

In [14]:
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 读取数据
hsi_data = pd.read_csv('hsi all shares data 20250613.csv', encoding='gbk')
stock_codes_df = pd.read_csv('stock_codes_full (1).csv')

# 获取医疗保健业股票代码(约200支)
medical_codes = stock_codes_df["医疗保健业"].dropna().tolist()

# 2. 初步筛选 - 放宽条件
# 在原始数据中获取这些股票的估值数据
filtered_data = hsi_data[hsi_data['WindCodes'].isin(medical_codes)].copy()

# 基本质量筛选(保留约150-180支)
filtered_data = filtered_data[
    (filtered_data['PE_TTM'] > 0) &  # 剔除亏损企业
    (filtered_data['PE_TTM'] < 150) &  # 放宽PE上限
    (filtered_data['MKT_CAP'] > 100000000) &  # 降低市值要求至1亿港币
    (filtered_data['MKT_FREESHARES'] > 100000000)  # 降低自由流通市值至1亿港币
]

# 3. 增长性筛选(保留约100-120支)
# 计算增长指标
filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'].pct_change()
filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'].pct_change()

# 保留增长为正或高于行业中位数的股票
sales_median = filtered_data['SALES_GROWTH'].median()
profit_median = filtered_data['NET_PROFIT_GROWTH'].median()

filtered_data = filtered_data[
    (filtered_data['SALES_GROWTH'] > sales_median*0.7) &  # 放宽增长要求
    (filtered_data['NET_PROFIT_GROWTH'] > profit_median*0.7) &
    (filtered_data['WGSD_EBIT_OPER'] > 0)  # 营业利润为正
]

# 4. 流动性筛选(保留约80-100支)
# 获取过去30天成交量数据
try:
    data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)
    volume = data['Volume'].mean(axis=0)
    volume_rank = volume.rank(pct=True)
    # 保留成交量在前80%的股票
    liquid_codes = volume_rank[volume_rank > 0.2].index.tolist()
    filtered_data = filtered_data[filtered_data['WindCodes'].isin(liquid_codes)]
except:
    print("未能获取成交量数据，跳过流动性筛选")

# 5. 综合评分(最终筛选40-60支)
# 计算综合评分(考虑估值、增长、规模等因素)
filtered_data['VALUATION_SCORE'] = 1/filtered_data['PE_TTM']  # PE倒数，越低越好
filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
filtered_data['SIZE_SCORE'] = np.log(filtered_data['MKT_CAP'])  # 对数转换

# 标准化各项指标
for col in ['VALUATION_SCORE', 'GROWTH_SCORE', 'SIZE_SCORE']:
    filtered_data[col] = (filtered_data[col] - filtered_data[col].mean()) / filtered_data[col].std()

# 加权综合评分(可根据偏好调整权重)
filtered_data['TOTAL_SCORE'] = (
    0.4 * filtered_data['VALUATION_SCORE'] +
    0.4 * filtered_data['GROWTH_SCORE'] +
    0.2 * filtered_data['SIZE_SCORE']
)

# 按总分排序
filtered_data = filtered_data.sort_values('TOTAL_SCORE', ascending=False)

# 确定最终数量(40-60支)
target_count = min(60, max(40, int(len(filtered_data)*0.3)))  # 取30%或40-60支
final_codes1 = filtered_data.head(target_count)['WindCodes'].tolist()

print(f"最终筛选出 {len(final_codes1)} 支医疗保健业股票:")
print(final_codes1)

# 保存结果
filtered_data.head(target_count).to_csv('selected_medical_stocks.csv', index=False, encoding='utf-8-sig')

/tmp/ipython-input-14-3717029756.py:26: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'].pct_change()
/tmp/ipython-input-14-3717029756.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'].pct_change()
/tmp/ipython-input-14-3717029756.py:42: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)


最终筛选出 35 支医疗保健业股票:
['3933.HK', '2607.HK', '3692.HK', '2666.HK', '1099.HK', '3320.HK', '0512.HK', '2196.HK', '2359.HK', '0241.HK', '6618.HK', '0874.HK', '1177.HK', '2005.HK', '6821.HK', '1513.HK', '0867.HK', '1066.HK', '0719.HK', '3759.HK', '6896.HK', '9989.HK', '1666.HK', '1093.HK', '1061.HK', '2269.HK', '1530.HK', '1358.HK', '2268.HK', '2186.HK', '3347.HK', '2172.HK', '1873.HK', '1858.HK', '6955.HK']


/tmp/ipython-input-14-3717029756.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['VALUATION_SCORE'] = 1/filtered_data['PE_TTM']  # PE倒数，越低越好
/tmp/ipython-input-14-3717029756.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
/tmp/ipython-input-14-3717029756.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [15]:
print("医疗")
print(final_codes1)

医疗
['3933.HK', '2607.HK', '3692.HK', '2666.HK', '1099.HK', '3320.HK', '0512.HK', '2196.HK', '2359.HK', '0241.HK', '6618.HK', '0874.HK', '1177.HK', '2005.HK', '6821.HK', '1513.HK', '0867.HK', '1066.HK', '0719.HK', '3759.HK', '6896.HK', '9989.HK', '1666.HK', '1093.HK', '1061.HK', '2269.HK', '1530.HK', '1358.HK', '2268.HK', '2186.HK', '3347.HK', '2172.HK', '1873.HK', '1858.HK', '6955.HK']


In [16]:
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 读取数据
hsi_data = pd.read_csv('hsi all shares data 20250613.csv', encoding='gbk')
stock_codes_df = pd.read_csv('stock_codes_full (1).csv')

# 获取资讯科技业股票代码
tech_codes = stock_codes_df["资讯科技业"].dropna().tolist()
print(f"资讯科技业初始股票数量: {len(tech_codes)}")

# 2. 初步筛选
filtered_data = hsi_data[hsi_data['WindCodes'].isin(tech_codes)].copy()

# 基本质量筛选(大幅放宽条件)
filtered_data = filtered_data[
    (filtered_data['PE_TTM'] > 0) &  # 仍剔除亏损企业
    (filtered_data['PE_TTM'] < 300) &  # 进一步放宽PE上限
    (filtered_data['MKT_CAP'] > 30000000) &  # 市值要求降至3000万港币
    (filtered_data['MKT_FREESHARES'] > 30000000)  # 自由流通市值降至3000万港币
]
print(f"基本质量筛选后剩余: {len(filtered_data)}")

# 3. 改进增长性筛选方法
# 使用绝对增长而非相对增长
filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'].diff()  # 使用绝对差值
filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'].diff()

# 计算行业增长阈值(使用更宽松的20分位数)
sales_growth_threshold = filtered_data['SALES_GROWTH'].quantile(0.2)
profit_growth_threshold = filtered_data['NET_PROFIT_GROWTH'].quantile(0.2)

# 修正的筛选条件 - 确保括号正确匹配
filtered_data = filtered_data[
    ((filtered_data['SALES_GROWTH'] > sales_growth_threshold) |
     (filtered_data['NET_PROFIT_GROWTH'] > profit_growth_threshold)) &
    (filtered_data['WGSD_EBIT_OPER'] > 0)
]
print(f"改进增长性筛选后剩余: {len(filtered_data)}")

# 4. 流动性筛选(调整为保留前80%)
try:
    data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)
    volume = data['Volume'].mean(axis=0)
    volume_rank = volume.rank(pct=True)
    liquid_codes = volume_rank[volume_rank > 0.2].index.tolist()  # 保留前80%
    filtered_data = filtered_data[filtered_data['WindCodes'].isin(liquid_codes)]
    print(f"流动性筛选后剩余: {len(filtered_data)}")
except Exception as e:
    print(f"未能获取成交量数据，跳过流动性筛选. 错误: {e}")

# 5. 综合评分与最终筛选
# 计算各项评分
filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
filtered_data['SIZE_SCORE'] = np.log1p(filtered_data['MKT_CAP'])
filtered_data['LIQUIDITY_SCORE'] = np.log1p(filtered_data['MKT_FREESHARES'])

# 标准化
for col in ['VALUATION_SCORE', 'GROWTH_SCORE', 'SIZE_SCORE', 'LIQUIDITY_SCORE']:
    filtered_data[col] = (filtered_data[col] - filtered_data[col].mean()) / filtered_data[col].std()

# 加权评分
filtered_data['TOTAL_SCORE'] = (
    0.25 * filtered_data['VALUATION_SCORE'] +
    0.45 * filtered_data['GROWTH_SCORE'] +
    0.15 * filtered_data['SIZE_SCORE'] +
    0.15 * filtered_data['LIQUIDITY_SCORE']
)

# 按总分排序
filtered_data = filtered_data.sort_values('TOTAL_SCORE', ascending=False)

# 动态确定最终数量(60-80支)
base_count = len(filtered_data)
if base_count <= 80:
    final_codes2 = filtered_data['WindCodes'].tolist()
else:
    target_count = min(80, max(60, int(base_count * 0.7)))
    final_codes2 = filtered_data.head(target_count)['WindCodes'].tolist()

print(f"\n最终筛选出 {len(final_codes2)} 支资讯科技业股票:")
print(final_codes2)

# 保存结果
result_df = filtered_data[filtered_data['WindCodes'].isin(final_codes2)]
result_df.to_csv('selected_tech_stocks.csv', index=False, encoding='utf-8-sig')

/tmp/ipython-input-16-3649526906.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)


资讯科技业初始股票数量: 233
基本质量筛选后剩余: 89
改进增长性筛选后剩余: 63
流动性筛选后剩余: 51

最终筛选出 51 支资讯科技业股票:
['0700.HK', '1810.HK', '3396.HK', '9999.HK', '0552.HK', '0285.HK', '0992.HK', '9698.HK', '3969.HK', '2660.HK', '1522.HK', '0856.HK', '1617.HK', '0327.HK', '0799.HK', '0696.HK', '3991.HK', '3888.HK', '6088.HK', '6869.HK', '1263.HK', '2400.HK', '0354.HK', '0434.HK', '1675.HK', '1686.HK', '0777.HK', '0596.HK', '6608.HK', '2166.HK', '1385.HK', '0536.HK', '1729.HK', '1720.HK', '8062.HK', '6069.HK', '1523.HK', '6860.HK', '0522.HK', '2878.HK', '1460.HK', '9636.HK', '2416.HK', '3315.HK', '2149.HK', '3738.HK', '1985.HK', '1059.HK', '1147.HK', '8017.HK', '3601.HK']


/tmp/ipython-input-16-3649526906.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
/tmp/ipython-input-16-3649526906.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
/tmp/ipython-input-16-3649526906.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [17]:
print("资讯科技")
print(final_codes2)

资讯科技
['0700.HK', '1810.HK', '3396.HK', '9999.HK', '0552.HK', '0285.HK', '0992.HK', '9698.HK', '3969.HK', '2660.HK', '1522.HK', '0856.HK', '1617.HK', '0327.HK', '0799.HK', '0696.HK', '3991.HK', '3888.HK', '6088.HK', '6869.HK', '1263.HK', '2400.HK', '0354.HK', '0434.HK', '1675.HK', '1686.HK', '0777.HK', '0596.HK', '6608.HK', '2166.HK', '1385.HK', '0536.HK', '1729.HK', '1720.HK', '8062.HK', '6069.HK', '1523.HK', '6860.HK', '0522.HK', '2878.HK', '1460.HK', '9636.HK', '2416.HK', '3315.HK', '2149.HK', '3738.HK', '1985.HK', '1059.HK', '1147.HK', '8017.HK', '3601.HK']


In [18]:
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 读取数据
hsi_data = pd.read_csv('hsi all shares data 20250613.csv', encoding='gbk')
stock_codes_df = pd.read_csv('stock_codes_full (1).csv')

# 获取资讯科技业股票代码
tech_codes = stock_codes_df["金融业"].dropna().tolist()
print(f"资讯科技业初始股票数量: {len(tech_codes)}")

# 2. 初步筛选
filtered_data = hsi_data[hsi_data['WindCodes'].isin(tech_codes)].copy()

# 基本质量筛选(大幅放宽条件)
filtered_data = filtered_data[
    (filtered_data['PE_TTM'] > 0) &  # 仍剔除亏损企业
    (filtered_data['PE_TTM'] < 300) &  # 进一步放宽PE上限
    (filtered_data['MKT_CAP'] > 30000000) &  # 市值要求降至3000万港币
    (filtered_data['MKT_FREESHARES'] > 30000000)  # 自由流通市值降至3000万港币
]
print(f"基本质量筛选后剩余: {len(filtered_data)}")

# 3. 改进增长性筛选方法
# 使用绝对增长而非相对增长
filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'].diff()  # 使用绝对差值
filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'].diff()

# 计算行业增长阈值(使用更宽松的20分位数)
sales_growth_threshold = filtered_data['SALES_GROWTH'].quantile(0.2)
profit_growth_threshold = filtered_data['NET_PROFIT_GROWTH'].quantile(0.2)

# 修正的筛选条件 - 确保括号正确匹配
filtered_data = filtered_data[
    ((filtered_data['SALES_GROWTH'] > sales_growth_threshold) |
     (filtered_data['NET_PROFIT_GROWTH'] > profit_growth_threshold)) &
    (filtered_data['WGSD_EBIT_OPER'] > 0)
]
print(f"改进增长性筛选后剩余: {len(filtered_data)}")

# 4. 流动性筛选(调整为保留前80%)
try:
    data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)
    volume = data['Volume'].mean(axis=0)
    volume_rank = volume.rank(pct=True)
    liquid_codes = volume_rank[volume_rank > 0.2].index.tolist()  # 保留前80%
    filtered_data = filtered_data[filtered_data['WindCodes'].isin(liquid_codes)]
    print(f"流动性筛选后剩余: {len(filtered_data)}")
except Exception as e:
    print(f"未能获取成交量数据，跳过流动性筛选. 错误: {e}")

# 5. 综合评分与最终筛选
# 计算各项评分
filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
filtered_data['SIZE_SCORE'] = np.log1p(filtered_data['MKT_CAP'])
filtered_data['LIQUIDITY_SCORE'] = np.log1p(filtered_data['MKT_FREESHARES'])

# 标准化
for col in ['VALUATION_SCORE', 'GROWTH_SCORE', 'SIZE_SCORE', 'LIQUIDITY_SCORE']:
    filtered_data[col] = (filtered_data[col] - filtered_data[col].mean()) / filtered_data[col].std()

# 加权评分
filtered_data['TOTAL_SCORE'] = (
    0.25 * filtered_data['VALUATION_SCORE'] +
    0.45 * filtered_data['GROWTH_SCORE'] +
    0.15 * filtered_data['SIZE_SCORE'] +
    0.15 * filtered_data['LIQUIDITY_SCORE']
)

# 按总分排序
filtered_data = filtered_data.sort_values('TOTAL_SCORE', ascending=False)

# 动态确定最终数量(60-80支)
base_count = len(filtered_data)
if base_count <= 80:
    final_codes3 = filtered_data['WindCodes'].tolist()
else:
    target_count = min(80, max(60, int(base_count * 0.7)))
    final_codes3 = filtered_data.head(target_count)['WindCodes'].tolist()

print(f"\n最终筛选出 {len(final_codes3)} 支资讯科技业股票:")
print(final_codes3)

# 保存结果
result_df = filtered_data[filtered_data['WindCodes'].isin(final_codes3)]
result_df.to_csv('selected_tech_stocks.csv', index=False, encoding='utf-8-sig')

资讯科技业初始股票数量: 231
基本质量筛选后剩余: 129
改进增长性筛选后剩余: 91


/tmp/ipython-input-18-3819970534.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)


流动性筛选后剩余: 73

最终筛选出 73 支资讯科技业股票:
['2318.HK', '1398.HK', '0939.HK', '1288.HK', '2628.HK', '1339.HK', '3988.HK', '2601.HK', '3968.HK', '1658.HK', '3328.HK', '1336.HK', '3698.HK', '1508.HK', '6818.HK', '0998.HK', '1988.HK', '9668.HK', '2388.HK', '1578.HK', '2016.HK', '1299.HK', '1606.HK', '2888.HK', '9889.HK', '6881.HK', '3618.HK', '6886.HK', '1216.HK', '6963.HK', '1963.HK', '1776.HK', '1905.HK', '0388.HK', '0011.HK', '6099.HK', '0440.HK', '6806.HK', '6066.HK', '1601.HK', '2378.HK', '6196.HK', '3958.HK', '6178.HK', '2139.HK', '1916.HK', '0900.HK', '6060.HK', '2858.HK', '1456.HK', '0717.HK', '0376.HK', '0245.HK', '0227.HK', '2596.HK', '0086.HK', '0697.HK', '1697.HK', '1141.HK', '1375.HK', '1476.HK', '1709.HK', '9923.HK', '1140.HK', '0863.HK', '1073.HK', '0628.HK', '8030.HK', '0111.HK', '8350.HK', '1217.HK', '1572.HK', '1461.HK']


/tmp/ipython-input-18-3819970534.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
/tmp/ipython-input-18-3819970534.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
/tmp/ipython-input-18-3819970534.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [19]:
print("金融73")
print(final_codes3)

金融73
['2318.HK', '1398.HK', '0939.HK', '1288.HK', '2628.HK', '1339.HK', '3988.HK', '2601.HK', '3968.HK', '1658.HK', '3328.HK', '1336.HK', '3698.HK', '1508.HK', '6818.HK', '0998.HK', '1988.HK', '9668.HK', '2388.HK', '1578.HK', '2016.HK', '1299.HK', '1606.HK', '2888.HK', '9889.HK', '6881.HK', '3618.HK', '6886.HK', '1216.HK', '6963.HK', '1963.HK', '1776.HK', '1905.HK', '0388.HK', '0011.HK', '6099.HK', '0440.HK', '6806.HK', '6066.HK', '1601.HK', '2378.HK', '6196.HK', '3958.HK', '6178.HK', '2139.HK', '1916.HK', '0900.HK', '6060.HK', '2858.HK', '1456.HK', '0717.HK', '0376.HK', '0245.HK', '0227.HK', '2596.HK', '0086.HK', '0697.HK', '1697.HK', '1141.HK', '1375.HK', '1476.HK', '1709.HK', '9923.HK', '1140.HK', '0863.HK', '1073.HK', '0628.HK', '8030.HK', '0111.HK', '8350.HK', '1217.HK', '1572.HK', '1461.HK']


In [20]:
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 读取数据
hsi_data = pd.read_csv('hsi all shares data 20250613.csv', encoding='gbk')
stock_codes_df = pd.read_csv('stock_codes_full (1).csv')

# 获取资讯科技业股票代码
tech_codes = stock_codes_df["非必需性消费"].dropna().tolist()
print(f"资讯科技业初始股票数量: {len(tech_codes)}")

# 2. 初步筛选
filtered_data = hsi_data[hsi_data['WindCodes'].isin(tech_codes)].copy()

# 基本质量筛选(大幅放宽条件)
filtered_data = filtered_data[
    (filtered_data['PE_TTM'] > 0) &  # 仍剔除亏损企业
    (filtered_data['PE_TTM'] < 100) &  # 进一步放宽PE上限
    (filtered_data['MKT_CAP'] > 50000000) &  # 市值要求降至3000万港币
    (filtered_data['MKT_FREESHARES'] > 50000000)  # 自由流通市值降至3000万港币
]
print(f"基本质量筛选后剩余: {len(filtered_data)}")

# 3. 改进增长性筛选方法
# 使用绝对增长而非相对增长
filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'].diff()  # 使用绝对差值
filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'].diff()

# 计算行业增长阈值(使用更宽松的20分位数)
sales_growth_threshold = filtered_data['SALES_GROWTH'].quantile(0.2)
profit_growth_threshold = filtered_data['NET_PROFIT_GROWTH'].quantile(0.2)

# 修正的筛选条件 - 确保括号正确匹配
filtered_data = filtered_data[
    ((filtered_data['SALES_GROWTH'] > sales_growth_threshold) |
     (filtered_data['NET_PROFIT_GROWTH'] > profit_growth_threshold)) &
    (filtered_data['WGSD_EBIT_OPER'] > 0)
]
print(f"改进增长性筛选后剩余: {len(filtered_data)}")

# 4. 流动性筛选(调整为保留前80%)
try:
    data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)
    volume = data['Volume'].mean(axis=0)
    volume_rank = volume.rank(pct=True)
    liquid_codes = volume_rank[volume_rank > 0.2].index.tolist()  # 保留前80%
    filtered_data = filtered_data[filtered_data['WindCodes'].isin(liquid_codes)]
    print(f"流动性筛选后剩余: {len(filtered_data)}")
except Exception as e:
    print(f"未能获取成交量数据，跳过流动性筛选. 错误: {e}")

# 5. 综合评分与最终筛选
# 计算各项评分
filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
filtered_data['SIZE_SCORE'] = np.log1p(filtered_data['MKT_CAP'])
filtered_data['LIQUIDITY_SCORE'] = np.log1p(filtered_data['MKT_FREESHARES'])

# 标准化
for col in ['VALUATION_SCORE', 'GROWTH_SCORE', 'SIZE_SCORE', 'LIQUIDITY_SCORE']:
    filtered_data[col] = (filtered_data[col] - filtered_data[col].mean()) / filtered_data[col].std()

# 加权评分
filtered_data['TOTAL_SCORE'] = (
    0.25 * filtered_data['VALUATION_SCORE'] +
    0.45 * filtered_data['GROWTH_SCORE'] +
    0.15 * filtered_data['SIZE_SCORE'] +
    0.15 * filtered_data['LIQUIDITY_SCORE']
)

# 按总分排序
filtered_data = filtered_data.sort_values('TOTAL_SCORE', ascending=False)

# 动态确定最终数量(60-80支)
base_count = len(filtered_data)
if base_count <= 80:
    final_codes4 = filtered_data['WindCodes'].tolist()
else:
    target_count = min(80, max(60, int(base_count * 0.7)))
    final_codes4 = filtered_data.head(target_count)['WindCodes'].tolist()

print(f"\n最终筛选出 {len(final_codes4)} 支资讯科技业股票:")
print(final_codes4)

# 保存结果
result_df = filtered_data[filtered_data['WindCodes'].isin(final_codes4)]
result_df.to_csv('selected_tech_stocks.csv', index=False, encoding='utf-8-sig')

资讯科技业初始股票数量: 677
基本质量筛选后剩余: 256
改进增长性筛选后剩余: 199


/tmp/ipython-input-20-542514964.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)


流动性筛选后剩余: 160

最终筛选出 80 支资讯科技业股票:
['9618.HK', '9988.HK', '1211.HK', '3690.HK', '0300.HK', '6690.HK', '0175.HK', '2559.HK', '1756.HK', '2333.HK', '9888.HK', '1024.HK', '2015.HK', '0881.HK', '0293.HK', '9961.HK', '0066.HK', '1958.HK', '1765.HK', '8512.HK', '1929.HK', '0921.HK', '1698.HK', '0027.HK', '2313.HK', '1070.HK', '3606.HK', '0669.HK', '0564.HK', '0382.HK', '1928.HK', '1809.HK', '0551.HK', '0425.HK', '2588.HK', '6862.HK', '3998.HK', '6169.HK', '1179.HK', '2282.HK', '1880.HK', '2779.HK', '2331.HK', '1999.HK', '2020.HK', '1128.HK', '1913.HK', '1910.HK', '1563.HK', '9901.HK', '0780.HK', '6110.HK', '0811.HK', '2111.HK', '1836.HK', '1797.HK', '0179.HK', '0116.HK', '1361.HK', '0751.HK', '1585.HK', '6181.HK', '6969.HK', '3813.HK', '2678.HK', '9899.HK', '1969.HK', '1368.HK', '2232.HK', '9616.HK', '0525.HK', '3306.HK', '1357.HK', '1571.HK', '1126.HK', '2298.HK', '1525.HK', '1318.HK', '1982.HK', '1773.HK']


/tmp/ipython-input-20-542514964.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
/tmp/ipython-input-20-542514964.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
/tmp/ipython-input-20-542514964.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [21]:
print("非必需消费品80")
print(final_codes4)

非必需消费品80
['9618.HK', '9988.HK', '1211.HK', '3690.HK', '0300.HK', '6690.HK', '0175.HK', '2559.HK', '1756.HK', '2333.HK', '9888.HK', '1024.HK', '2015.HK', '0881.HK', '0293.HK', '9961.HK', '0066.HK', '1958.HK', '1765.HK', '8512.HK', '1929.HK', '0921.HK', '1698.HK', '0027.HK', '2313.HK', '1070.HK', '3606.HK', '0669.HK', '0564.HK', '0382.HK', '1928.HK', '1809.HK', '0551.HK', '0425.HK', '2588.HK', '6862.HK', '3998.HK', '6169.HK', '1179.HK', '2282.HK', '1880.HK', '2779.HK', '2331.HK', '1999.HK', '2020.HK', '1128.HK', '1913.HK', '1910.HK', '1563.HK', '9901.HK', '0780.HK', '6110.HK', '0811.HK', '2111.HK', '1836.HK', '1797.HK', '0179.HK', '0116.HK', '1361.HK', '0751.HK', '1585.HK', '6181.HK', '6969.HK', '3813.HK', '2678.HK', '9899.HK', '1969.HK', '1368.HK', '2232.HK', '9616.HK', '0525.HK', '3306.HK', '1357.HK', '1571.HK', '1126.HK', '2298.HK', '1525.HK', '1318.HK', '1982.HK', '1773.HK']


In [22]:
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 读取数据
hsi_data = pd.read_csv('hsi all shares data 20250613.csv', encoding='gbk')
stock_codes_df = pd.read_csv('stock_codes_full (1).csv')

# 获取资讯科技业股票代码
tech_codes = stock_codes_df["工业"].dropna().tolist()
print(f"资讯科技业初始股票数量: {len(tech_codes)}")

# 2. 初步筛选
filtered_data = hsi_data[hsi_data['WindCodes'].isin(tech_codes)].copy()

# 基本质量筛选(大幅放宽条件)
filtered_data = filtered_data[
    (filtered_data['PE_TTM'] > 0) &  # 仍剔除亏损企业
    (filtered_data['PE_TTM'] < 100) &  # 进一步放宽PE上限
    (filtered_data['MKT_CAP'] > 50000000) &  # 市值要求降至3000万港币
    (filtered_data['MKT_FREESHARES'] > 50000000)  # 自由流通市值降至3000万港币
]
print(f"基本质量筛选后剩余: {len(filtered_data)}")

# 3. 改进增长性筛选方法
# 使用绝对增长而非相对增长
filtered_data['SALES_GROWTH'] = filtered_data['WGSD_SALES'].diff()  # 使用绝对差值
filtered_data['NET_PROFIT_GROWTH'] = filtered_data['WGSD_NET_INC'].diff()

# 计算行业增长阈值(使用更宽松的20分位数)
sales_growth_threshold = filtered_data['SALES_GROWTH'].quantile(0.2)
profit_growth_threshold = filtered_data['NET_PROFIT_GROWTH'].quantile(0.2)

# 修正的筛选条件 - 确保括号正确匹配
filtered_data = filtered_data[
    ((filtered_data['SALES_GROWTH'] > sales_growth_threshold) |
     (filtered_data['NET_PROFIT_GROWTH'] > profit_growth_threshold)) &
    (filtered_data['WGSD_EBIT_OPER'] > 0)
]
print(f"改进增长性筛选后剩余: {len(filtered_data)}")

# 4. 流动性筛选(调整为保留前80%)
try:
    data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)
    volume = data['Volume'].mean(axis=0)
    volume_rank = volume.rank(pct=True)
    liquid_codes = volume_rank[volume_rank > 0.3].index.tolist()  # 保留前80%
    filtered_data = filtered_data[filtered_data['WindCodes'].isin(liquid_codes)]
    print(f"流动性筛选后剩余: {len(filtered_data)}")
except Exception as e:
    print(f"未能获取成交量数据，跳过流动性筛选. 错误: {e}")

# 5. 综合评分与最终筛选
# 计算各项评分
filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
filtered_data['SIZE_SCORE'] = np.log1p(filtered_data['MKT_CAP'])
filtered_data['LIQUIDITY_SCORE'] = np.log1p(filtered_data['MKT_FREESHARES'])

# 标准化
for col in ['VALUATION_SCORE', 'GROWTH_SCORE', 'SIZE_SCORE', 'LIQUIDITY_SCORE']:
    filtered_data[col] = (filtered_data[col] - filtered_data[col].mean()) / filtered_data[col].std()

# 加权评分
filtered_data['TOTAL_SCORE'] = (
    0.25 * filtered_data['VALUATION_SCORE'] +
    0.45 * filtered_data['GROWTH_SCORE'] +
    0.15 * filtered_data['SIZE_SCORE'] +
    0.15 * filtered_data['LIQUIDITY_SCORE']
)

# 按总分排序
filtered_data = filtered_data.sort_values('TOTAL_SCORE', ascending=False)

# 动态确定最终数量(60-80支)
base_count = len(filtered_data)
if base_count <= 80:
    final_codes5 = filtered_data['WindCodes'].tolist()
else:
    target_count = min(80, max(60, int(base_count * 0.7)))
    final_codes5 = filtered_data.head(target_count)['WindCodes'].tolist()

print(f"\n最终筛选出 {len(final_codes5)} 支资讯科技业股票:")
print(final_codes5)

# 保存结果
result_df = filtered_data[filtered_data['WindCodes'].isin(final_codes5)]
result_df.to_csv('selected_tech_stocks.csv', index=False, encoding='utf-8-sig')

资讯科技业初始股票数量: 324
基本质量筛选后剩余: 151
改进增长性筛选后剩余: 116


/tmp/ipython-input-22-1544166795.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(filtered_data['WindCodes'].tolist(), period='30d', progress=False)


流动性筛选后剩余: 82

最终筛选出 60 支资讯科技业股票:
['3750.HK', '3996.HK', '1919.HK', '6936.HK', '0269.HK', '2338.HK', '2618.HK', '2039.HK', '0598.HK', '2727.HK', '2357.HK', '2382.HK', '1072.HK', '1157.HK', '0819.HK', '0316.HK', '0144.HK', '2018.HK', '0177.HK', '1308.HK', '3898.HK', '6198.HK', '2208.HK', '0576.HK', '1519.HK', '0868.HK', '1882.HK', '0148.HK', '0586.HK', '0636.HK', '1888.HK', '1138.HK', '3931.HK', '6865.HK', '0548.HK', '1415.HK', '2276.HK', '0107.HK', '2510.HK', '1478.HK', '9956.HK', '0951.HK', '3369.HK', '0152.HK', '9930.HK', '3339.HK', '2155.HK', '0995.HK', '0968.HK', '2507.HK', '1052.HK', '3393.HK', '1133.HK', '1651.HK', '0732.HK', '3382.HK', '0558.HK', '1330.HK', '1286.HK', '0842.HK']


/tmp/ipython-input-22-1544166795.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['VALUATION_SCORE'] = 1/np.log1p(filtered_data['PE_TTM'])
/tmp/ipython-input-22-1544166795.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['GROWTH_SCORE'] = filtered_data['SALES_GROWTH'] + filtered_data['NET_PROFIT_GROWTH']
/tmp/ipython-input-22-1544166795.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [23]:
print("工业60")
print(final_codes5)

工业60
['3750.HK', '3996.HK', '1919.HK', '6936.HK', '0269.HK', '2338.HK', '2618.HK', '2039.HK', '0598.HK', '2727.HK', '2357.HK', '2382.HK', '1072.HK', '1157.HK', '0819.HK', '0316.HK', '0144.HK', '2018.HK', '0177.HK', '1308.HK', '3898.HK', '6198.HK', '2208.HK', '0576.HK', '1519.HK', '0868.HK', '1882.HK', '0148.HK', '0586.HK', '0636.HK', '1888.HK', '1138.HK', '3931.HK', '6865.HK', '0548.HK', '1415.HK', '2276.HK', '0107.HK', '2510.HK', '1478.HK', '9956.HK', '0951.HK', '3369.HK', '0152.HK', '9930.HK', '3339.HK', '2155.HK', '0995.HK', '0968.HK', '2507.HK', '1052.HK', '3393.HK', '1133.HK', '1651.HK', '0732.HK', '3382.HK', '0558.HK', '1330.HK', '1286.HK', '0842.HK']


In [29]:
# 假设已经定义了五个列表 final_codes1, final_codes2, final_codes3, final_codes4, final_codes5

# 合并五个列表
merged_codes = final_codes1 + final_codes2 + final_codes3 + final_codes4 + final_codes5

# 打印合并后的列表
print("合并后的完整列表:")
print(merged_codes)

# 打印每个列表的内容
print("\n各个列表的内容:")
print(f"医疗: {final_codes1}")
print(f"资讯科技: {final_codes2}")
print(f"金融: {final_codes3}")
print(f"非必需消费品: {final_codes4}")
print(f"工业: {final_codes5}")
print(f"一共有{len(merged_codes)}支")


合并后的完整列表:
['3933.HK', '2607.HK', '3692.HK', '2666.HK', '1099.HK', '3320.HK', '0512.HK', '2196.HK', '2359.HK', '0241.HK', '6618.HK', '0874.HK', '1177.HK', '2005.HK', '6821.HK', '1513.HK', '0867.HK', '1066.HK', '0719.HK', '3759.HK', '6896.HK', '9989.HK', '1666.HK', '1093.HK', '1061.HK', '2269.HK', '1530.HK', '1358.HK', '2268.HK', '2186.HK', '3347.HK', '2172.HK', '1873.HK', '1858.HK', '6955.HK', '0700.HK', '1810.HK', '3396.HK', '9999.HK', '0552.HK', '0285.HK', '0992.HK', '9698.HK', '3969.HK', '2660.HK', '1522.HK', '0856.HK', '1617.HK', '0327.HK', '0799.HK', '0696.HK', '3991.HK', '3888.HK', '6088.HK', '6869.HK', '1263.HK', '2400.HK', '0354.HK', '0434.HK', '1675.HK', '1686.HK', '0777.HK', '0596.HK', '6608.HK', '2166.HK', '1385.HK', '0536.HK', '1729.HK', '1720.HK', '8062.HK', '6069.HK', '1523.HK', '6860.HK', '0522.HK', '2878.HK', '1460.HK', '9636.HK', '2416.HK', '3315.HK', '2149.HK', '3738.HK', '1985.HK', '1059.HK', '1147.HK', '8017.HK', '3601.HK', '2318.HK', '1398.HK', '0939.HK', '1288.HK',